In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tqdm
import pandas as pd

In [ ]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
# font_location = '/usr/share/fonts/truetype/nanum/NanumGothicOTF.ttf'
font_location = 'C:/Windows/Fonts/NanumGothicCoding.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

%matplotlib inline

In [ ]:
import os
cwd = os.getcwd()

In [ ]:
from typing import Tuple, List
Vector = List[float]

In [ ]:
import time
now_time = 1610366047.3755555
to_do = "함수 least_squares_fit을 작성하기 위해 stats 파일에 있는 기초함수를 구현한다"
page = 202

#### 21-01-13

In [ ]:
os.chdir('C:/Users/jongh/OneDrive/python/machine-learning/Data-Science-from-Scratch-master/code-python3')
from stats import correlation, standard_deviation, mean, num_friends_good, daily_minutes_good, de_mean
from gradient_descent import step
os.chdir(cwd)

In [ ]:
x = [i for i in range(-100, 110, 10)]
y = [3*i-5 for i in x]

In [ ]:
def least_square_fit(x:Vector, y:Vector)->Tuple[float, float]:
    beta = correlation(x, y) * standard_deviation(y) / standard_deviation(x)
    alpha = mean(y) - beta * mean(x)
    return alpha, beta

assert least_square_fit(x, y) == (-5, 3)

In [ ]:
_alpha, _beta = least_square_fit(num_friends_good, daily_minutes_good)

In [ ]:
def predict(alpha, beta, x_i):
    x_i = np.array(x_i)
    return beta * x_i + alpha

In [ ]:
_predict = predict(_alpha, _beta, num_friends_good)
# _predict

In [ ]:
plt.title('최소자승법(least squares fit)을 이용한 사용시간 예측')
plt.plot(num_friends_good, _predict)
plt.scatter(num_friends_good, daily_minutes_good)
plt.show()

In [ ]:
def error(alpha, beta, x_i, y_i):
    y_i = np.array(y_i)
    return predict(alpha, beta, x_i) - y_i

In [ ]:
_error = error(_alpha, _beta, num_friends_good, daily_minutes_good)

In [ ]:
_df = pd.DataFrame(_error, columns=['error'])
# category = pd.cut(_df['error'], 30)
_df['error'].hist(bins=20)
plt.show()

In [ ]:
def sum_of_sqerrors(alpha, beta, x, y):
    _error = error(_alpha, _beta, x, y)    
    return sum(_error ** 2)

In [ ]:
def total_sum_of_squares(y):
    y = np.array(y)
    y_bar = np.mean(y)
    return sum((y - y_bar) ** 2)

In [ ]:
sse = sum_of_sqerrors(_alpha, _beta, num_friends_good, daily_minutes_good)
sst = total_sum_of_squares(daily_minutes_good)
length = len(daily_minutes_good)
_mean = np.array(daily_minutes_good).mean()

In [ ]:
def r_squared(alpha: float, beta: float, x: Vector, y: Vector) -> float:
    """
    모델이 잡아낼 있는 y의 변화량의 비율은
    1 - 모델이 잡아내지 못하는 y의 변화량의 비율로 계산할 수 있다.
    """
    sse = sum_of_sqerrors(alpha, beta, x, y)
    sst = total_sum_of_squares(y)
    return 1 - (sse/sst)

In [ ]:
rsq = r_squared(_alpha, _beta, num_friends_good, daily_minutes_good)
rsq

In [ ]:
y_bar = np.array(daily_minutes_good).mean()

In [ ]:
(sum(error(_alpha, _beta, num_friends_good, daily_minutes_good) ** 2) / length) ** (1/2)

In [ ]:
plt.scatter(num_friends_good, daily_minutes_good)
_x = np.arange(-0, 55)
plt.plot(_x, _x * _beta + _alpha, c='orange')
plt.title("using time depending on num of friends")
plt.xlabel("num_friends_good")
plt.ylabel("daily_minutes_good")

plt.hlines(_mean, 0, 50,  colors='r')
plt.legend(['prediction', 'real data', 'mean-y'])

plt.show()

## 14.2 경사 하강법 사용하기

In [ ]:
random.seed(0)
guess = [random.random(), random.random()]
lr = 1e-5
num_epochs = 10000

alpha, beta = guess
/print guess

loss = sum_of_sqerrors(alpha, beta, num_friends_good, daily_minutes_good)

with tqdm.trange(num_epochs) as t:
    for _ in t:
        alpha,beta = guess
        grad_a = sum(2 * error(alpha, beta, num_friends_good, daily_minutes_good))
        
        grad_b = sum(2*error(alpha, beta, num_friends_good, daily_minutes_good)*num_friends_good)
        
        loss = sum_of_sqerrors(alpha, beta, num_friends_good, daily_minutes_good)
        t.set_description(f"loss: {loss:.3f}")
        
        guess = step(guess, [grad_a, grad_b], -lr)

alpha, beta = guess
alpha, beta

### https://brunch.co.kr/@zhoyp/169

In [ ]:
index = ['철수', '영희', '순이', '갑돌', '기영']
columns = ['키', '몸무게']
sta_df = pd.DataFrame(index=index, columns=columns)

In [ ]:
sta_df['키'] = [100, 90, 105, 110, 95]
sta_df['몸무게'] = [50, 45, 60, 55, 40]
sta_df

In [ ]:
height_mean = sta_df['키'].mean()
weight_mean = sta_df['몸무게'].mean()

In [ ]:
height_std = sta_df['키'].std(ddof=0)
weight_std = sta_df['몸무게'].std(ddof=0)

In [ ]:
sta_df['stndard_height'] = (sta_df['키'] - height_mean) / height_std

In [ ]:
sta_df['stndard_weight'] = (sta_df['몸무게'] - weight_mean) / weight_std

In [ ]:
sta_df['standard_height*weight'] = sta_df['stndard_height'] * sta_df['stndard_weight']

In [ ]:
sta_df

In [ ]:
상관계수 = sta_df['standard_height*weight'].mean()
상관계수

### https://www.youtube.com/watch?v=XhC1RQimW-g

In [ ]:
os.chdir('C:/Users/jongh/OneDrive/python/machine-learning/data')
housedata = pd.read_csv('kc_house_data.csv')
os.chdir(cwd)

In [ ]:
housedata.columns

In [ ]:
plt.scatter(housedata['sqft_living'], housedata['price']/1000)
plt.show()